# Packet Delivery Agent

In [1]:
import random
import math
import numpy as np

### Subset generation(weights) using simulated annealing

In [91]:
# Method to generate random sample from an array for weight packets
def generate_random_solution(weights, length):
    return sorted(random.sample(weights, length))


# Method to generate all possible neighbours
def get_neighbourhood_subsets(weights, subset):
    neighbourhood = []
    
    # Neighbours with one element less
    if len(subset) > 1:
        for index in range(len(subset)):
            neighbourhood.append(subset[:index] + subset[index + 1:])

    # Neighbours with one element more
    if len(subset) < len(weights):
        for item in weights:
            if item not in subset:
                neighbourhood.append(subset + [item])
    
    # Replace every element in subset with one element from set
    for index in range(len(subset)):
        for set_item in weights:
            if set_item not in subset:
                neighbourhood.append(subset[:index] + [set_item] + subset[index + 1:])
    return neighbourhood


# Method to determine whether random_subset is better than
# normal subset. Returns True if random_subset is better, False otherwise
def is_better_neighbour(subset, random_subset):
    if random_subset:
        return abs(sum(random_subset)) < abs(sum(subset))
    return False

# Method for performing the simulated_annealing for generating subset which has sum 10
def simulated_annealing(target_sum, weights, subset, best_subset, initial_temp, final_temp, max_iterations, cooling_ratio, iterations):
    if sum(subset) == target_sum:
        return best_subset
    if initial_temp <= final_temp:
        return best_subset
    if iterations == max_iterations:
        return best_subset
    if len(subset) == 1:
        return subset
    
    neighbourhood_subsets = get_neighbourhood_subsets(weights, subset)
    # print(f"Neighbours: {neighbourhood_subsets}")
    # print(f"Total number of Neighbours: {len(neighbourhood_subsets)}")
    
    random_neighbour = random.choice(neighbourhood_subsets)
    # print(f"Random neighbour chosen: {random_neighbour} of sum: {sum(random_neighbour)}")
    
    if is_better_neighbour(subset, random_neighbour):
        subset = random_neighbour
        best_subset = subset
    else:
        r = random.uniform(0, 1)
        val = abs(sum(random_neighbour) - sum(subset)) / float(initial_temp)
        if r < math.exp(-val):
            subset = random_neighbour
    
    iterations += 1
    initial_temp = cooling_ratio * initial_temp
    
    return simulated_annealing(target_sum, weights, subset, best_subset, initial_temp, final_temp, max_iterations, cooling_ratio, iterations)

### Local beam search algorithm with heuristic on rooms - packet distribution

In [92]:
# Function to generate a single random successor state from a given  state .Is called by the generate_1000 function
def generate_one(st, packet_vec):
    st_size = len(st)
    for i in packet_vec:
        while True:
            loc=np.random.randint(0, st_size)
            if st[loc] >= i:
                st[loc] = st[loc] - i
                break
    return st


# Generates 10000 successors and returns the unique successors calls the generate_one 10000 times
def generate_10000(state,packet_vec):
    a = []
    for i in range(1000):
        a.append(generate_one(state.copy(),packet_vec))
    return set([tuple(i) for i in a]) ## only returns unique states 


# Finds the best state based on the generated random states and the free_space, 
# chooses the one which optimises the most effective space consumed
def best_State(state_collection, rooms, updated_room_state):
    current_fitness_value = 0
    initial = rooms # [5, 28, 20, 46, 25, 35]
    prority_order = np.arange(len(rooms), 0, -1) # prority given to biggest room
    #print(f"prority_order: {prority_order}")
    for curr_state in state_collection:
        diff = np.array(updated_room_state) - np.array(curr_state)
        mul = np.array(diff)*np.array(prority_order)
        fitness_value=sum(mul)
        # print(f"curr_state: {curr_state} :: diff: {diff} :: fitness_value: {fitness_value}")
        if fitness_value > current_fitness_value:
            current_fitness_value = fitness_value
            locally_optimal_state = curr_state
    return locally_optimal_state

# Helper method to if there is any wrong weight input
def check_is_sufficient(st, packet_vec):
    if len(st) == 0:
        max_room = 0
    else:
        max_room = max(st)
    for packet in packet_vec:
        if(packet >= max_room):
            return False
    return True

# Local beam search algorithm
def local_Search_Rooms(packet_string, current_state, rooms):
    is_sufficient = check_is_sufficient(current_state, packet_string)
    if not is_sufficient:
        return []
    all_generated_successors = generate_10000(current_state, packet_string)
    best_state = best_State(all_generated_successors, rooms, current_state)
    return best_state

### Packet Delivery Agent driver code

In [93]:
# Helper function to print the summary of the packet delivery agent
def print_summary(weight_map, max_weight_robot_can_carry, rooms, keyword='end', commutes=0):
    footer = f"Total commutes that robot has taken {commutes}"
    if keyword == 'start':
        footer = f"Maximum weight that the robot can carry: {max_weight_robot_can_carry}"
    print(f"""
--------------------------------------------------
        Packet Delivery Agent {keyword} summary
--------------------------------------------------
    Weight map: {weight_map}
    Rooms: {rooms}
    {footer}
---------------------------------------------------
""")


# Helper method to generate list from weight map
def generate_list_from_weight_map(weight_map):
    final = []
    for key in weight_map.keys():
        if weight_map[key] != 0:
            final.extend([key] * weight_map[key])
    return final

# Driver code to run packet delivery agent. Check below for the usage
def packet_delivery_agent(weight_map, max_weight_robot_can_carry, rooms):
    total_no_of_commutes = 0
    rooms.sort(reverse=True)
    
    # Constants for simulated annealing
    max_iterations = 500
    cooling_ratio = 0.99
    final_temp = 0.01
    old_temp = 1500
    
    target_sum = max_weight_robot_can_carry # For example:- 10
    
    room_states = rooms
    print_summary(weight_map, max_weight_robot_can_carry, rooms, 'start')
    
    while(len(list(filter(lambda x: weight_map[x] > 0, list(weight_map.keys())))) > 0):
        weights = generate_list_from_weight_map(weight_map)

        # Generating a random subset
        subset = generate_random_solution(weights, random.randint(1, len(weights)))
        best_subset = subset
        
        temp_subset = subset
        temp_best_subset = subset
        
        iterations = 1
        
        initial_temp = 1500
        
        final_subset = simulated_annealing(target_sum, weights, subset, best_subset, initial_temp, final_temp, max_iterations, cooling_ratio, iterations)

        # Robot should carry the specified maximum weight
        if(sum(final_subset) > max_weight_robot_can_carry):
            final_subset = simulated_annealing(target_sum, weights, temp_subset, temp_best_subset, old_temp, final_temp, max_iterations, cooling_ratio, iterations)

        print(f"Selected Weight packet: {final_subset} and sum: {sum(final_subset)}")
        updated_state = local_Search_Rooms(final_subset, room_states, rooms)
        print(f"Updated Room state: {updated_state}")
        
        if(len(updated_state) ==0):# If weight doesnot fit in any room
            print(f"weight packets: {final_subset} doesnot fit in any room {room_states}")
            break
        
        for item in final_subset:
            weight_map[item] -= 1

        room_states = list(updated_state)

        print(f"Updated weight map is: {weight_map}")
        total_no_of_commutes += 1
        print(f"--------Commutes count : {total_no_of_commutes}-----------")
    
    print_summary(weight_map, max_weight_robot_can_carry, room_states, commutes=total_no_of_commutes)

### Code Usage and Run

In [94]:
##########################################################################
"""
    Packet Delivery Agent code Usage
________________________________________

1. Initialize your weight map 
   For Example:- weight_map = {1: 2, 5: 2, 4: 2, 6: 2, 9: 2}
2. Decide on the maximum weight that the robot can carry in kgs
   For Example:- max_weight_robot_can_carry = 10
3. Decide on the rooms that are present
   For Example:- rooms = [25, 45, 36] 

4. Finally call the packet_delivery_agent method like this:-
   packet_delivery_agent(weight_map, max_weight_robot_can_carry, rooms)
   
   Test data => weight_carried = 10
   1. rooms = [9, 2, 5] , weight_map = {1: 1, 9:1}
   2. rooms = [8, 8, 8], weight_map = {1: 3, 7:2, 3:1}
   3. rooms = [8, 20, 5], weight_map = {7:2, 9:2}
   4. rooms = [], weight_map = {}
   5. rooms = [2, 5], weight_map = {}
   6. rooms = [], weight_map = {1: 2, 9: 2}
"""
##########################################################################

# Code run goes here:- 
rooms = [5, 28, 20, 46, 25, 35]
weight_map = {1: 15, 2: 5, 4: 7, 5: 2, 6: 3, 7: 2, 9: 4}
weight_carried = 10
packet_delivery_agent(weight_map, weight_carried, rooms)


--------------------------------------------------
        Packet Delivery Agent start summary
--------------------------------------------------
    Weight map: {1: 15, 2: 5, 4: 7, 5: 2, 6: 3, 7: 2, 9: 4}
    Rooms: [46, 35, 28, 25, 20, 5]
    Maximum weight that the robot can carry: 10
---------------------------------------------------

Selected Weight packet: [2, 1, 7] and sum: 10
Updated Room state: (36, 35, 28, 25, 20, 5)
Updated weight map is: {1: 14, 2: 4, 4: 7, 5: 2, 6: 3, 7: 1, 9: 4}
--------Commutes count : 1-----------
Selected Weight packet: [2, 1, 7] and sum: 10
Updated Room state: (26, 35, 28, 25, 20, 5)
Updated weight map is: {1: 13, 2: 3, 4: 7, 5: 2, 6: 3, 7: 0, 9: 4}
--------Commutes count : 2-----------
Selected Weight packet: [1, 9] and sum: 10
Updated Room state: (16, 35, 28, 25, 20, 5)
Updated weight map is: {1: 12, 2: 3, 4: 7, 5: 2, 6: 3, 7: 0, 9: 3}
--------Commutes count : 3-----------
Selected Weight packet: [1, 4] and sum: 5
Updated Room state: (11, 35, 28, 